In [ ]:
import seaborn as sns
import pandas as pd
from pyspark.sql import SparkSession

# Cargar datos 
tips = sns.load_dataset('tips')

# Iniciando sesión de Spark
spark = SparkSession.builder.appName("Data Cleansing with PySpark and SQL").getOrCreate()

# De pandas a PySpark DataFrame
df = spark.createDataFrame(tips)
df.createOrReplaceTempView("tabla_tips")  # Crear vista temporal en PySpark SQL
df.show(5)

In [ ]:
## Eliminar duplicados
result = spark.sql("SELECT DISTINCT * FROM tabla_tips")
result.show()

In [ ]:
## Con PySpark usamos dropDuplicates() para eliminar duplicados.

df_no_duplicates = df.dropDuplicates()
df_no_duplicates.show(5)

In [ ]:
## Contar valores nulos en cada columna
from pyspark.sql.functions import count, when, isnull
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

In [ ]:
## Eliminar filas con valores nulos en las columnas 'total_bill' y 'tip'
from pyspark.sql.functions import col
result = spark.sql("SELECT * FROM tabla_tips WHERE total_bill IS NOT NULL AND tip IS NOT NULL")
result.show()

In [ ]:
## Reemplazar valores nulos en la columna 'tip' con 0
from pyspark.sql.functions import coalesce
result = spark.sql("SELECT COALESCE(tip, 0) AS tip, total_bill, sex, smoker, day, time, size FROM tabla_tips")
result.show()

In [ ]:
## Reemplazar valores nulos en la columna 'tip' con 0 usando PySpark
df_filled = df.fillna({'tip': 0})
df_filled.show(5)

In [ ]:
## Filtrar datos: seleccionar filas donde 'size' es mayor que 2 y 'total_bill' es mayor que 20
result = spark.sql("SELECT * FROM tabla_tips WHERE size > 2 AND total_bill > 20")
result.show()

In [ ]:
## Filtrar registros en PySpark sin usar SQL.
df_filtered = df.filter((df['size'] > 2) & (df['total_bill'] > 20))
df_filtered.show(5)

In [ ]:
## Calcular el porcentaje de propina respecto a la cuenta total
result = spark.sql("SELECT *, (tip / total_bill) * 100 AS tip_percentage FROM tabla_tips")
result.show()

In [ ]:
## Calcular el porcentaje de propina respecto a la cuenta total usando PySpark
from pyspark.sql.functions import col

df = df.withColumn("tip_percentage", (df["tip"] / df["total_bill"]) * 100)
df.show(5)

In [ ]:
## Agrupar por día y calcular la propina promedio
result = spark.sql("SELECT day, AVG(tip) AS avg_tip FROM tabla_tips GROUP BY day")
result.show()

In [ ]:
## Agrupar por día y calcular la propina promedio usando PySpark
df_agg = df.groupBy("day").agg({"tip": "avg"})
df_agg.show()

In [ ]:
import matplotlib.pyplot as plt

# Cargar el dataset 'tips' de Seaborn
tips = sns.load_dataset('tips')

# Crear una figura con varios subgráficos
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Gráfico 1: Distribución continua (total_bill)
sns.histplot(tips['total_bill'], kde=True, ax=axes[0, 0], color='blue')
axes[0, 0].set_title('Distribución Continua: Total Bill')

# Gráfico 2: Distribución discreta (size)
sns.histplot(tips['size'], kde=False, ax=axes[0, 1], color='green', discrete=True)
axes[0, 1].set_title('Distribución Discreta: Size')

# Gráfico 3: Boxplot para detectar outliers (tip)
sns.boxplot(x=tips['tip'], ax=axes[1, 0], color='purple')
axes[1, 0].set_title('Boxplot: Tip')

# Gráfico 4: Gráfico de violín para distribución por día
sns.violinplot(x="day", y="total_bill", data=tips, ax=axes[1, 1], palette="muted")
axes[1, 1].set_title('Gráfico de Violín: Total Bill por Día')

# Ajustar el diseño para que no se solapen los gráficos
plt.tight_layout()
plt.show()
